# Match Analysis

In [1]:
import pandas as pd
import json
import numpy as np

from kaggle_environments.envs.football.helpers import Action

pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 150)

np.set_printoptions(suppress=True)

TO DO - Set Default Dictionary for actions, sticky actions, etc..

In [2]:
# dictionary of sticky actions
sticky_actions = {
    "left": Action.Left,
    "top_left": Action.TopLeft,
    "top": Action.Top,
    "top_right": Action.TopRight,
    "right": Action.Right,
    "bottom_right": Action.BottomRight,
    "bottom": Action.Bottom,
    "bottom_left": Action.BottomLeft,
    "sprint": Action.Sprint,
    "dribble": Action.Dribble
}

action_set_dic = {
    0: 'idle',
    # movement actions (1-8)
    1: 'left', # run to the left, sticky action.
    2: 'top_left', #run to the top-left, sticky action.
    3: 'top', # run to the top, sticky action.
    4: 'top_right', # run to the top-right, sticky action.
    5: 'right', # run to the right, sticky action.
    6: 'bottom_right', # run to the bottom-right, sticky action.
    7: 'bottom', # run to the bottom, sticky action.
    8: 'bottom_left', # run to the bottom-left, sticky action
    
    # passing / shooting (9-12)
    9: 'long_pass', # perform a long pass to the player on your team. Player to pass the ball to is auto-determined based on the movement direction.
    10: 'high_pass', # perform a high pass, similar to long_pass.
    11: 'short_pass', # perform a short pass, similar to long_pass.
    12: 'shot', # perform a shot, always in the direction of the opponent's goal.
    
    13: 'sprint', # start sprinting, sticky action. Player moves faster, but has worse ball handling.
    14: 'release_direction', # reset current movement direction.
    15: 'release_sprint', # stop sprinting
    16: 'sliding', # perform a slide (effective when not having a ball)
    17: 'dribble', # start dribbling (effective when having a ball), sticky action. Player moves slower, but it is harder to take over the ball from him.
    18: 'release_dribble' # stop dribbling
}


# game_mode - current game mode, one of:
game_mode_dic = {
    0 : 'normal',
    1 : 'kickoff',
    2 : 'goalkick',
    3 : 'freekick',
    4 : 'corner',
    5 : 'throwin',
    6 : 'penalty'
}

# player roles
player_role_dic = {
    0: 'GK',
    1: 'CB',
    2: 'LB',
    3: 'RB',
    4: 'DM',
    5: 'CM',
    6: 'LM',
    7: 'RM',
    8: 'AM',
    9: 'CF'
}

## DataFrame Create

## Get simple115_v2 embed df

In [4]:
# tdf = get_simple115_v2_df(obs)
# print(tdf.shape)

In [5]:
# tdf.head(2)

In [6]:
team_want = 'WeKick'
episode_id = 3953996
episode_dir = '../episode_scraping/episodes_dl/WeKick/28-10-2020-05-36-35/'

tdf = get_episode_simple115_v2_df(team_want, episode_id, episode_dir)
print(tdf.shape)

(3002, 115)


In [7]:
tdf.head(2)

,l_x0,l_y0,l_x1,l_y1,l_x2,l_y2,l_x3,l_y3,l_x4,l_y4,l_x5,l_y5,l_x6,l_y6,l_x7,l_y7,l_x8,l_y8,l_x9,l_y9,l_x10,l_y10,l_x_dir0,l_y_dir0,l_x_dir1,l_y_dir1,l_x_dir2,l_y_dir2,l_x_dir3,l_y_dir3,l_x_dir4,l_y_dir4,l_x_dir5,l_y_dir5,l_x_dir6,l_y_dir6,l_x_dir7,l_y_dir7,l_x_dir8,l_y_dir8,l_x_dir9,l_y_dir9,l_x_dir10,l_y_dir10,r_x0,r_y0,r_x1,r_y1,r_x2,r_y2,r_x3,r_y3,r_x4,r_y4,r_x5,r_y5,r_x6,r_y6,r_x7,r_y7,r_x8,r_y8,r_x9,r_y9,r_x10,r_y10,r_x_dir0,r_y_dir0,r_x_dir1,r_y_dir1,r_x_dir2,r_y_dir2,r_x_dir3,r_y_dir3,r_x_dir4,r_y_dir4,r_x_dir5,r_y_dir5,r_x_dir6,r_y_dir6,r_x_dir7,r_y_dir7,r_x_dir8,r_y_dir8,r_x_dir9,r_y_dir9,r_x_dir10,r_y_dir10,ball_x_pos,ball_y_pos,ball_z_pos,ball_x_dir,ball_y_dir,ball_z_dir,ball_own_noone,ball_own_left,ball_own_right,p_active0,p_active1,p_active2,p_active3,p_active4,p_active5,p_active6,p_active7,p_active8,p_active9,p_active10,gmode0,gmode1,gmode2,gmode3,gmode4,gmode5,gmode6
0,-1.01103,-0.00000,-0.05055,-0.00000,-0.01011,0.21962,-0.42665,-0.19894,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.01011,-0.21962,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,1.01103,0.00000,-0.00000,-0.02033,-0.00000,0.02033,0.42665,0.19894,0.50551,0.06459,0.50551,-0.06459,0.42665,-0.19894,0.18624,0.10740,0.27053,0.00000,0.18624,-0.10740,0.01011,0.21962,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,-0.00000,0.11062,0.00000,-0.00000,0.00616,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,-1.01103,-0.00000,-0.05122,-0.00047,-0.01011,0.21960,-0.42533,-0.19827,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.00917,-0.21901,0.00000,-0.00000,-0.00218,-0.00156,-0.00000,-0.00003,0.00278,0.00146,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00240,0.00159,1.01096,-0.00000,-0.00000,-0.01899,-0.00005,0.02036,0.42665,0.19894,0.50548,0.06456,0.50551,-0.06459,0.42498,-0.19818,0.18624,0.10740,0.27053,0.00000,0.18463,-0.10657,0.01010,0.21939,-0.00066,-0.00000,-0.00000,0.00245,-0.00047,0.00030,-0.00000,0.00000,-0.00037,-0.00036,-0.00000,0.00000,-0.00307,0.00139,-0.00000,0.00000,-0.00000,0.00000,-0.00305,0.00147,-0.00007,-0.00122,0.00000,-0.00000,0.11060,0.00000,-0.00000,-0.00192,1.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [8]:
# get json file for specific episode
epsiode_full_dir = '{0}{1}.json'.format(episode_dir, episode_id)
epsiode_full_dir

with open(epsiode_full_dir) as json_file:
    obs = json.load(json_file)

In [9]:
lteam = obs['info']['TeamNames'][0]
rteam = obs['info']['TeamNames'][1]

In [10]:
print(tdf.shape)

(3002, 115)


In [11]:
tdf.head(2)

,l_x0,l_y0,l_x1,l_y1,l_x2,l_y2,l_x3,l_y3,l_x4,l_y4,l_x5,l_y5,l_x6,l_y6,l_x7,l_y7,l_x8,l_y8,l_x9,l_y9,l_x10,l_y10,l_x_dir0,l_y_dir0,l_x_dir1,l_y_dir1,l_x_dir2,l_y_dir2,l_x_dir3,l_y_dir3,l_x_dir4,l_y_dir4,l_x_dir5,l_y_dir5,l_x_dir6,l_y_dir6,l_x_dir7,l_y_dir7,l_x_dir8,l_y_dir8,l_x_dir9,l_y_dir9,l_x_dir10,l_y_dir10,r_x0,r_y0,r_x1,r_y1,r_x2,r_y2,r_x3,r_y3,r_x4,r_y4,r_x5,r_y5,r_x6,r_y6,r_x7,r_y7,r_x8,r_y8,r_x9,r_y9,r_x10,r_y10,r_x_dir0,r_y_dir0,r_x_dir1,r_y_dir1,r_x_dir2,r_y_dir2,r_x_dir3,r_y_dir3,r_x_dir4,r_y_dir4,r_x_dir5,r_y_dir5,r_x_dir6,r_y_dir6,r_x_dir7,r_y_dir7,r_x_dir8,r_y_dir8,r_x_dir9,r_y_dir9,r_x_dir10,r_y_dir10,ball_x_pos,ball_y_pos,ball_z_pos,ball_x_dir,ball_y_dir,ball_z_dir,ball_own_noone,ball_own_left,ball_own_right,p_active0,p_active1,p_active2,p_active3,p_active4,p_active5,p_active6,p_active7,p_active8,p_active9,p_active10,gmode0,gmode1,gmode2,gmode3,gmode4,gmode5,gmode6
0,-1.01103,-0.00000,-0.05055,-0.00000,-0.01011,0.21962,-0.42665,-0.19894,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.01011,-0.21962,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,1.01103,0.00000,-0.00000,-0.02033,-0.00000,0.02033,0.42665,0.19894,0.50551,0.06459,0.50551,-0.06459,0.42665,-0.19894,0.18624,0.10740,0.27053,0.00000,0.18624,-0.10740,0.01011,0.21962,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,-0.00000,0.11062,0.00000,-0.00000,0.00616,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,-1.01103,-0.00000,-0.05122,-0.00047,-0.01011,0.21960,-0.42533,-0.19827,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.00917,-0.21901,0.00000,-0.00000,-0.00218,-0.00156,-0.00000,-0.00003,0.00278,0.00146,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00240,0.00159,1.01096,-0.00000,-0.00000,-0.01899,-0.00005,0.02036,0.42665,0.19894,0.50548,0.06456,0.50551,-0.06459,0.42498,-0.19818,0.18624,0.10740,0.27053,0.00000,0.18463,-0.10657,0.01010,0.21939,-0.00066,-0.00000,-0.00000,0.00245,-0.00047,0.00030,-0.00000,0.00000,-0.00037,-0.00036,-0.00000,0.00000,-0.00307,0.00139,-0.00000,0.00000,-0.00000,0.00000,-0.00305,0.00147,-0.00007,-0.00122,0.00000,-0.00000,0.11060,0.00000,-0.00000,-0.00192,1.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [12]:
tdf = get_episode_all_df(team_want, episode_id, episode_dir)
print(tdf.shape)

(3002, 121)


In [13]:
tdf.head(2)

,step_num,action_str,game_mode_str,active_player,ball_owned_team,status,l_x0,l_y0,l_x1,l_y1,l_x2,l_y2,l_x3,l_y3,l_x4,l_y4,l_x5,l_y5,l_x6,l_y6,l_x7,l_y7,l_x8,l_y8,l_x9,l_y9,l_x10,l_y10,l_x_dir0,l_y_dir0,l_x_dir1,l_y_dir1,l_x_dir2,l_y_dir2,l_x_dir3,l_y_dir3,l_x_dir4,l_y_dir4,l_x_dir5,l_y_dir5,l_x_dir6,l_y_dir6,l_x_dir7,l_y_dir7,l_x_dir8,l_y_dir8,l_x_dir9,l_y_dir9,l_x_dir10,l_y_dir10,r_x0,r_y0,r_x1,r_y1,r_x2,r_y2,r_x3,r_y3,r_x4,r_y4,r_x5,r_y5,r_x6,r_y6,r_x7,r_y7,r_x8,r_y8,r_x9,r_y9,r_x10,r_y10,r_x_dir0,r_y_dir0,r_x_dir1,r_y_dir1,r_x_dir2,r_y_dir2,r_x_dir3,r_y_dir3,r_x_dir4,r_y_dir4,r_x_dir5,r_y_dir5,r_x_dir6,r_y_dir6,r_x_dir7,r_y_dir7,r_x_dir8,r_y_dir8,r_x_dir9,r_y_dir9,r_x_dir10,r_y_dir10,ball_x_pos,ball_y_pos,ball_z_pos,ball_x_dir,ball_y_dir,ball_z_dir,ball_own_noone,ball_own_left,ball_own_right,p_active0,p_active1,p_active2,p_active3,p_active4,p_active5,p_active6,p_active7,p_active8,p_active9,p_active10,gmode0,gmode1,gmode2,gmode3,gmode4,gmode5,gmode6
0,0,None,normal,6,-1,ACTIVE,-1.01103,-0.00000,-0.05055,-0.00000,-0.01011,0.21962,-0.42665,-0.19894,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.01011,-0.21962,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,1.01103,0.00000,-0.00000,-0.02033,-0.00000,0.02033,0.42665,0.19894,0.50551,0.06459,0.50551,-0.06459,0.42665,-0.19894,0.18624,0.10740,0.27053,0.00000,0.18624,-0.10740,0.01011,0.21962,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,-0.00000,0.11062,0.00000,-0.00000,0.00616,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,1,top_left,normal,1,-1,ACTIVE,-1.01103,-0.00000,-0.05122,-0.00047,-0.01011,0.21960,-0.42533,-0.19827,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.00917,-0.21901,0.00000,-0.00000,-0.00218,-0.00156,-0.00000,-0.00003,0.00278,0.00146,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00240,0.00159,1.01096,-0.00000,-0.00000,-0.01899,-0.00005,0.02036,0.42665,0.19894,0.50548,0.06456,0.50551,-0.06459,0.42498,-0.19818,0.18624,0.10740,0.27053,0.00000,0.18463,-0.10657,0.01010,0.21939,-0.00066,-0.00000,-0.00000,0.00245,-0.00047,0.00030,-0.00000,0.00000,-0.00037,-0.00036,-0.00000,0.00000,-0.00307,0.00139,-0.00000,0.00000,-0.00000,0.00000,-0.00305,0.00147,-0.00007,-0.00122,0.00000,-0.00000,0.11060,0.00000,-0.00000,-0.00192,1.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [14]:
import match_analysis_helper as mah

In [15]:
tdf = mah.get_episode_all_df(team_want, episode_id, episode_dir)

In [16]:
tdf.head(2)

,step_num,action_str,game_mode_str,active_player,ball_owned_team,status,l_x0,l_y0,l_x1,l_y1,l_x2,l_y2,l_x3,l_y3,l_x4,l_y4,l_x5,l_y5,l_x6,l_y6,l_x7,l_y7,l_x8,l_y8,l_x9,l_y9,l_x10,l_y10,l_x_dir0,l_y_dir0,l_x_dir1,l_y_dir1,l_x_dir2,l_y_dir2,l_x_dir3,l_y_dir3,l_x_dir4,l_y_dir4,l_x_dir5,l_y_dir5,l_x_dir6,l_y_dir6,l_x_dir7,l_y_dir7,l_x_dir8,l_y_dir8,l_x_dir9,l_y_dir9,l_x_dir10,l_y_dir10,r_x0,r_y0,r_x1,r_y1,r_x2,r_y2,r_x3,r_y3,r_x4,r_y4,r_x5,r_y5,r_x6,r_y6,r_x7,r_y7,r_x8,r_y8,r_x9,r_y9,r_x10,r_y10,r_x_dir0,r_y_dir0,r_x_dir1,r_y_dir1,r_x_dir2,r_y_dir2,r_x_dir3,r_y_dir3,r_x_dir4,r_y_dir4,r_x_dir5,r_y_dir5,r_x_dir6,r_y_dir6,r_x_dir7,r_y_dir7,r_x_dir8,r_y_dir8,r_x_dir9,r_y_dir9,r_x_dir10,r_y_dir10,ball_x_pos,ball_y_pos,ball_z_pos,ball_x_dir,ball_y_dir,ball_z_dir,ball_own_noone,ball_own_left,ball_own_right,p_active0,p_active1,p_active2,p_active3,p_active4,p_active5,p_active6,p_active7,p_active8,p_active9,p_active10,gmode0,gmode1,gmode2,gmode3,gmode4,gmode5,gmode6
0,0,None,normal,6,-1,ACTIVE,-1.01103,-0.00000,-0.05055,-0.00000,-0.01011,0.21962,-0.42665,-0.19894,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.01011,-0.21962,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,1.01103,0.00000,-0.00000,-0.02033,-0.00000,0.02033,0.42665,0.19894,0.50551,0.06459,0.50551,-0.06459,0.42665,-0.19894,0.18624,0.10740,0.27053,0.00000,0.18624,-0.10740,0.01011,0.21962,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,-0.00000,0.11062,0.00000,-0.00000,0.00616,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,1,top_left,normal,1,-1,ACTIVE,-1.01103,-0.00000,-0.05122,-0.00047,-0.01011,0.21960,-0.42533,-0.19827,-0.50551,-0.06459,-0.50551,0.06459,-0.42665,0.19894,-0.18624,-0.10740,-0.27053,-0.00000,-0.18624,0.10740,-0.00917,-0.21901,0.00000,-0.00000,-0.00218,-0.00156,-0.00000,-0.00003,0.00278,0.00146,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00240,0.00159,1.01096,-0.00000,-0.00000,-0.01899,-0.00005,0.02036,0.42665,0.19894,0.50548,0.06456,0.50551,-0.06459,0.42498,-0.19818,0.18624,0.10740,0.27053,0.00000,0.18463,-0.10657,0.01010,0.21939,-0.00066,-0.00000,-0.00000,0.00245,-0.00047,0.00030,-0.00000,0.00000,-0.00037,-0.00036,-0.00000,0.00000,-0.00307,0.00139,-0.00000,0.00000,-0.00000,0.00000,-0.00305,0.00147,-0.00007,-0.00122,0.00000,-0.00000,0.11060,0.00000,-0.00000,-0.00192,1.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
